In [ ]:
# Importing necessary libraries  Machine Learning for Predictive Analytics 
#a classifier that uses data to predict if a patient has diabetes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
import joblib
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from tabulate import tabulate
import numpy as np
import pandas as df 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder 
from mlxtend.frequent_patterns import apriori,association_rules
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
import time
from sklearn import preprocessing
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
#%pip install six
import six
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# plotly library
import plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
# word cloud library
from wordcloud import WordCloud
# matplotlib library
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
sys.modules['sklearn.externals.six'] = six
from sklearn.ensemble import RandomForestClassifier
import plotly as py
import plotly.graph_objs as go

In [ ]:
#Loading the data

df = pd.read_csv("diabetes.csv")

In [ ]:
styled_df = df.head(5).style

# Set background color, text color, and border for the entire DataFrame
styled_df.set_properties(**{"background-color": "#254E58", "color": "#e9c46a", "border": "1.5px solid black"})

# Modify the color and background color of the table headers (th)
styled_df.set_table_styles([
    {"selector": "th", "props": [("color", 'white'), ("background-color", "#333333")]}
])

3.2.2 | Number of rows, number of columns

In [ ]:
rows , col =  df.shape
print(f"Number of Rows : {rows} \nNumber of Columns : {col}")

In [ ]:
df.info()


In [ ]:
df.isnull().sum()


3.2.5 | Description

In [ ]:
styled_df = df.describe().style \
    .set_table_styles([
        {'selector': 'th', 'props': [('background-color', '#254E58'), ('color', 'white'), ('font-weight', 'bold'), ('text-align', 'left'), ('padding', '8px')]},
        {'selector': 'td', 'props': [('padding', '8px')]}
    ]) \
    .set_properties(**{'font-size': '14px', 'background-color': '#F5F5F5', 'border-collapse': 'collapse', 'margin': '10px'})

# Display the styled DataFrame
styled_df

In [ ]:
import missingno as msno
msno.bar(df)
plt.show()

In [ ]:
df.hist(figsize = (10,10))
plt.show()

3.3.2 | Boxplot

In [ ]:
num_rows, num_cols = 3, 3

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 10))

# Flatten the axes for easier iteration
axes = axes.flatten()

# Loop through numeric columns and create boxplots
for i, column in enumerate(df.columns):
    sns.boxplot(data=df, x=column, ax=axes[i])
    axes[i].set_title(f'Boxplot for {column}')

# Remove any remaining empty subplots
for j in range(len(df.columns), len(axes)):
    fig.delaxes(axes[j])

# Adjust layout
plt.tight_layout()
plt.show()

3.3.3 | Pairplot of attributes

In [ ]:
sns.pairplot(data = df, hue = 'Outcome' )
plt.show()

3.3.4 | Age and Outcome

In [ ]:
sns.set(rc={"axes.facecolor":"#EAE7F9","figure.facecolor":"#EAE7F9"})
p=sns.catplot(x="Outcome",y="Age", data=df, kind='box')
plt.title("Age and Outcome Correlation", size=20, y=1.0);

3.3.5 | Glucose and Outcome Correlation

In [ ]:
sns.set(rc={"axes.facecolor":"#EAE7F9","figure.facecolor":"#EAE7F9"})
p=sns.catplot(x="Outcome",y="Glucose", data=df, kind='box')
plt.title("Glucose and Outcome Correlation", size=20, y=1.0);

3.3.6 | Correlation between attributes

In [ ]:
plt.figure(figsize=(20, 17))
matrix = np.triu(df.corr())
sns.heatmap(df.corr(), annot=True, linewidth=.8, mask=matrix, cmap="rocket");

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(df.corr(), annot=True);

In [ ]:
hig_corr = df.corr()
hig_corr_features = hig_corr.index[abs(hig_corr["Outcome"]) >= 0.2]
hig_corr_features

3.3.7 | Standard Deviation

In [ ]:
#Standard Deviation
df.var()

 4 | Data Preprocessing


Handling Outliers

In [ ]:
numeric_columns = ['Insulin', 'DiabetesPedigreeFunction',]

for column_name in numeric_columns:
    Q1 = np.percentile(df[column_name], 25, interpolation='midpoint')
    Q3 = np.percentile(df[column_name], 75, interpolation='midpoint')

    IQR = Q3 - Q1
    low_lim = Q1 - 1.5 * IQR
    up_lim = Q3 + 1.5 * IQR

    # Find outliers in the specified column
    outliers = df[(df[column_name] < low_lim) | (df[column_name] > up_lim)][column_name]

    # Replace outliers with the respective lower or upper limit
    df[column_name] = np.where(df[column_name] < low_lim, low_lim, df[column_name])
    df[column_name] = np.where(df[column_name] > up_lim, up_lim, df[column_name])

Get `input` and `target` columns

In [ ]:
X = df.drop('Outcome', axis = 1)
y = df['Outcome']

In [ ]:
# Data for the bar plot (number of instances for each column)
data = {
    'Column': ['Index', 'Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
    'Count': [6136, 6136, 6136, 6136, 6136, 6136, 6136, 6136, 6136, 6136]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Plot the number of instances for each column
plt.figure(figsize=(10, 6))
plt.bar(df['Column'], df['Count'], color='b', alpha=0.7)
plt.xlabel('Column')
plt.ylabel('Number of Instances')
plt.title('Number of Instances for Each Column')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Sample data for the first three data entries
data = pd.DataFrame({
    'Pregnancies': [1, 8, 1],
    'Glucose': [85, 183, 89],
    'BloodPressure': [66, 64, 66],
    'SkinThickness': [29, 0, 23],
    'Insulin': [0, 0, 94],
    'BMI': [26.6, 23.3, 28.1],
    'DiabetesPedigreeFunction': [0.351, 0.672, 0.167],
    'Age': [31, 32, 21],
    'Outcome': [0, 1, 0]
})

# Print the first three data entries as a table using tabulate
print(tabulate(data.head(3), headers='keys', tablefmt='psql'))

In [ ]:
# to print the last ten data entries using head().
data.tail(3)

In [ ]:
# to get the features and its data type using info()
data.info()

In [ ]:
import pandas as pd

# Assuming 'data' is your dictionary containing the dataset
# Convert the dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(data)

# Obtain the data types of each column
print(df.dtypes)

In [ ]:

# Sample data 
data = {
    'Pregnancies': [6, 1, 8, 1, 0],
    'Glucose': [148, 85, 183, 89, 137],
    'BloodPressure': [72, 66, 64, 66, 40],
    'SkinThickness': [35, 29, 0, 23, 35],
    'Insulin': [0, 0, 0, 94, 168],
    'BMI': [33.6, 26.6, 23.3, 28.1, 43.1],
    'DiabetesPedigreeFunction': [0.627, 0.351, 0.672, 0.167, 2.288],
    'Age': [50, 31, 32, 21, 33],
    'Outcome': [1, 0, 1, 0, 1]
}

df = pd.DataFrame(data)

# Bar Plot - Count of Outcome Categories
outcome_counts = df['Outcome'].value_counts()
plt.bar(outcome_counts.index, outcome_counts.values)
plt.xlabel('Outcome')
plt.ylabel('Count')
plt.title('Count of Outcome Categories')
plt.xticks(outcome_counts.index, ['Not Diabetic', 'Diabetic'])
plt.show()

# Histogram - Age Distribution
plt.hist(df['Age'], bins=20, edgecolor='k')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Age Distribution')
plt.show()

# Scatter Plot - BMI vs. Glucose with Outcome Color-Coding
plt.scatter(df['BMI'], df['Glucose'], c=df['Outcome'], cmap='coolwarm', alpha=0.8)
plt.xlabel('BMI')
plt.ylabel('Glucose')
plt.title('BMI vs. Glucose with Outcome Color-Coding')
plt.colorbar(label='Outcome')
plt.show()

# Box Plot - Glucose Levels by Outcome
plt.boxplot([df[df['Outcome'] == 0]['Glucose'], df[df['Outcome'] == 1]['Glucose']], labels=['Not Diabetic', 'Diabetic'])
plt.ylabel('Glucose')
plt.title('Glucose Levels by Outcome')
plt.show()

# Correlation Heatmap
import seaborn as sns

correlation_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
import pandas as pd

# Assuming 'data' is your dictionary
data = {'column1': [1, 2, 3], 'column2': ['A', 'B', 'C']}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Now you can use the 'dtypes' attribute to get the data types of columns
data_types_counts = df.dtypes.value_counts()
print(data_types_counts)

In [ ]:
# Assuming 'data' is your dictionary
data = {'column1': [1, 2, 3], 'column2': ['A', 'B', 'C']}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Now you can use the methods and attributes on the DataFrame
# To obtain the summary statistics
print(df.describe())

# To obtain the count of each column
print(df.count())

# To obtain the values of each column
print(df.values)

In [ ]:
# Assuming 'data' is your dictionary
data = {'column1': [1, 2, 3, None], 'column2': ['A', 'B', None, 'C']}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Now you can use the method on the DataFrame to determine the presence of missing values
print(df.isna())

# Get the count of missing values in each column
print(df.isna().sum())


In [ ]:
# Assuming 'data' is your dictionary
data = {'column1': [1, 2, 3, None], 'column2': ['A', 'B', None, 'C']}

# Convert the dictionary to a DataFrame
df = pd.DataFrame(data)

# Now you can use the method on the DataFrame to drop rows with missing values
df_dropped = df.dropna()

# The variable 'df_dropped' now contains the DataFrame with rows containing missing values removed.
print(df_dropped)

In [ ]:
data = {
    'key1': [1, 2, 3, None, 5],
    'key2': [None, 'a', 'b', 'c', 'd']
}

# Determine the presence of missing values.
missing_values = {key: [value is None for value in values] for key, values in data.items()}
print(missing_values)

# Get the count of missing values.
missing_counts = {key: sum(value is None for value in values) for key, values in data.items()}
print(missing_counts)

In [ ]:
# Data for learning rate and accuracy scores
learning_rates = [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
training_scores = [0.767, 0.779, 0.795, 0.819, 0.834, 0.862, 0.857]
validation_scores = [0.739, 0.752, 0.771, 0.784, 0.784, 0.739, 0.771]

# Plotting the learning rate against the training and validation accuracy scores
plt.figure(figsize=(8, 6))
plt.plot(learning_rates, training_scores, marker='o', label='Training Accuracy')
plt.plot(learning_rates, validation_scores, marker='o', label='Validation Accuracy')
plt.xlabel('Learning Rate')
plt.ylabel('Accuracy Score')
plt.title('Learning Rate vs. Accuracy Score')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Data for accuracy scores
accuracy_scores = [1.000, 0.752]
labels = ['Training Accuracy', 'Validation Accuracy']

# Plotting the accuracy scores
plt.figure(figsize=(6, 4))
plt.bar(labels, accuracy_scores, color=['blue', 'orange'])
plt.ylabel('Accuracy Score')
plt.title('Training vs. Validation Accuracy')
plt.ylim(0, 1.1)
plt.grid(axis='y')
plt.show()

Splitting data for the training

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.20)

5 | Machine Learning models


5.1 | Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=1, penalty='l2', solver='liblinear', max_iter=200)
log_reg.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

def predict_and_plot(model, inputs, targets, name=''):
    preds = model.predict(inputs)
    accuracy = accuracy_score(targets, preds)
    print("Accuracy: {:.2f}%".format(accuracy * 100))
    
    cf = confusion_matrix(targets, preds, normalize='true')
    plt.figure()
    sns.heatmap(cf, annot=True)
    plt.xlabel('Prediction')
    plt.ylabel('Target')
    plt.title('{} Confusion Matrix'.format(name))
    
    return preds

# Predict and plot on the training data
train_preds = predict_and_plot(log_reg, X_train, y_train, 'Train')

# Predict and plot on the validation data
val_preds = predict_and_plot(log_reg, X_test, y_test, 'Validation')

5.2 | Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_2 = RandomForestClassifier(n_jobs =-1, random_state = 42)
model_2.fit(X_train,y_train)

In [ ]:
model_2.score(X_train,y_train)

In [ ]:
def predict_and_plot(model, inputs,targets, name = ''):
    preds = model.predict(inputs)
    accuracy = accuracy_score(targets, preds)
    print("Accuracy: {:.2f}%".format(accuracy*100))
    
    cf = confusion_matrix(targets, preds, normalize = 'true')
    plt.figure()
    sns.heatmap(cf, annot = True)
    plt.xlabel('Prediction')
    plt.ylabel('Target')
    plt.title('{} Confusion Matrix'. format(name))
    
    return preds

# Predict and plot on the training data
train_preds = predict_and_plot(model_2, X_train, y_train, 'Train')

# Predict and plot on the validation data
val_preds = predict_and_plot(model_2, X_test, y_test, 'Validation')

Hyperparameter Tuning of Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

param_grid = {
    'n_estimators': [10, 20, 30],  # Adjust the number of trees in the forest
    'max_depth': [10, 20, 30],  # Adjust the maximum depth of each tree
    'min_samples_split': [2, 5, 10, 15, 20],  # Adjust the minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4, 6, 8]  # Adjust the minimum samples required in a leaf node
}

model = RandomForestClassifier(random_state=42, n_jobs=-1)
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_

best_model.fit(X_train, y_train)

# Evaluate the model on the training and validation data
train_accuracy = best_model.score(X_train, y_train)
val_accuracy = best_model.score(X_test, y_test)

# Print the results
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

5.3 |Decision Tree


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Create the Decision Tree model
decision_tree_model = DecisionTreeClassifier(random_state=42)

# Fit the model to the training data
decision_tree_model.fit(X_train, y_train)

# Evaluate the model on the training and validation data
train_accuracy = decision_tree_model.score(X_train, y_train)
val_accuracy = decision_tree_model.score(X_test, y_test)

# Print the results
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

Hyperparameter Tuning Of Decision Tree



In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10, 15, 20, 25],
    'min_samples_leaf': [1, 3, 5, 7],
    'criterion': ['gini', 'entropy']  # Add criterion hyperparameter
}



# Create the Decision Tree model
decision_tree_model = DecisionTreeClassifier(random_state=42)

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(decision_tree_model, param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best estimator with tuned hyperparameters
best_model = grid_search.best_estimator_

# Fit the final model to the training data
best_model.fit(X_train, y_train)

# Evaluate the final model on the training and validation data
train_accuracy = best_model.score(X_train, y_train)
val_accuracy = best_model.score(X_test, y_test)

# Print the results
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

5.4 |KNeighborsClassifier Model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a KNN classifier (you can adjust the number of neighbors 'n_neighbors')
knn_model = KNeighborsClassifier(n_neighbors=5)

# Fit the KNN model to the training data
knn_model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred = knn_model.predict(X_train)

# Make predictions on the validation data
y_val_pred = knn_model.predict(X_val)

# Calculate the training and validation accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

# Create a confusion matrix for validation data
confusion = confusion_matrix(y_val, y_val_pred)

# Plot the confusion matrix
plt.figure(figsize=(6, 4))
sns.heatmap(confusion, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Validation)')
plt.show()


Hyperparameter Tuning of KNN



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the hyperparameter grid to search
param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9]  # Adjust the number of neighbors to explore
}

# Create the KNN classifier
knn_model = KNeighborsClassifier()

# Perform GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best estimator with tuned hyperparameters
best_model = grid_search.best_estimator_

# Make predictions on the training data using the best model
y_train_pred = best_model.predict(X_train)

# Make predictions on the validation data using the best model
y_val_pred = best_model.predict(X_val)

# Calculate the training and validation accuracies
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)

print("Training Accuracy with Best Hyperparameters:", train_accuracy)
print("Validation Accuracy with Best Hyperparameters:", val_accuracy)

5.5 |Support Vector Classifier

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

svm_model = SVC(kernel='linear')

# Fit the SVM model to the training data
svm_model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred = svm_model.predict(X_train)

# Make predictions on the validation data
y_val_pred = svm_model.predict(X_val)

# Calculate the training accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)

# Calculate the validation accuracy
val_accuracy = accuracy_score(y_val, y_val_pred)

print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

# Create confusion matrices
train_confusion = confusion_matrix(y_train, y_train_pred)
val_confusion = confusion_matrix(y_val, y_val_pred)

# Plot the confusion matrices
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.heatmap(train_confusion, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Training)')

plt.subplot(1, 2, 2)
sns.heatmap(val_confusion, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Validation)')
plt.show()

5.6 |AdaBoost Classifier

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Create an AdaBoost classifier
adaboost_model = AdaBoostClassifier(n_estimators=50, random_state=42)

# Fit the AdaBoost model to the training data
adaboost_model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred_adaboost = adaboost_model.predict(X_train)

# Make predictions on the validation data
y_val_pred_adaboost = adaboost_model.predict(X_val)

# Calculate the training accuracy
train_accuracy_adaboost = accuracy_score(y_train, y_train_pred_adaboost)

# Calculate the validation accuracy
val_accuracy_adaboost = accuracy_score(y_val, y_val_pred_adaboost)

# Print the training and validation accuracies
print("AdaBoost Training Accuracy:", train_accuracy_adaboost)
print("AdaBoost Validation Accuracy:", val_accuracy_adaboost)

# Create a confusion matrix for validation
confusion_adaboost = confusion_matrix(y_val, y_val_pred_adaboost)

# Plot the confusion matrix
plt.figure()
sns.heatmap(confusion_adaboost, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('AdaBoost Confusion Matrix (Validation)')
plt.show()

5.7 | Gradient Boosting Classifier


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Create a Gradient Boosting classifier
gbm_model = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)

# Fit the GBM model to the training data
gbm_model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred_gbm = gbm_model.predict(X_train)

# Make predictions on the validation data
y_val_pred_gbm = gbm_model.predict(X_val)

# Calculate the training accuracy
train_accuracy_gbm = accuracy_score(y_train, y_train_pred_gbm)

# Calculate the validation accuracy
val_accuracy_gbm = accuracy_score(y_val, y_val_pred_gbm)

# Print the training and validation accuracies
print("GBM Training Accuracy:", train_accuracy_gbm)
print("GBM Validation Accuracy:", val_accuracy_gbm)

5.8 | xgboost Classifier

In [ ]:
from xgboost import XGBClassifier

# Create an XGBoost classifier
xgboost_model = XGBClassifier(n_estimators=100, max_depth=3, random_state=42)

# Fit the XGBoost model to the training data
xgboost_model.fit(X_train, y_train)

# Make predictions on the training data
y_train_pred_xgboost = xgboost_model.predict(X_train)

# Make predictions on the validation data
y_val_pred_xgboost = xgboost_model.predict(X_val)

# Calculate the training accuracy
train_accuracy_xgboost = accuracy_score(y_train, y_train_pred_xgboost)

# Calculate the validation accuracy
val_accuracy_xgboost = accuracy_score(y_val, y_val_pred_xgboost)

# Print the training and validation accuracies
print("XGBoost Training Accuracy:", train_accuracy_xgboost)
print("XGBoost Validation Accuracy:", val_accuracy_xgboost)

In [ ]:
# Define the scores and models
# (Assuming you have already calculated the test scores for each algorithm)

# Replace the test scores below with the actual test scores you obtained from the code

Logisticregressionscoretest = 0.81
KNeighborsClassifierModelscoretest = 0.88
GaussianNBModelscoretest = 0.76
SVCModelscoretest = 0.87
DecisionTreeClassifierModeltest = 0.93
RandomForestClassifierModeltest = 0.89
gb_clftest = 0.86
xgb_test = 0.91

# Create the DataFrame
models = pd.DataFrame({
    'Model': ['Logistic Regression', 'KNN', 'Naive Bayes', 'Linear SVC', 'Decision Tree', 'Random Forest', 'Gradient Booster', 'XGBoost'],
    'Test Score': [Logisticregressionscoretest, KNeighborsClassifierModelscoretest, GaussianNBModelscoretest, SVCModelscoretest, DecisionTreeClassifierModeltest, RandomForestClassifierModeltest, gb_clftest, xgb_test]
})

# Plot the test scores
plt.figure(figsize=(10, 6))
plt.bar(models['Model'], models['Test Score'], color='b', alpha=0.7)
plt.xlabel('Model')
plt.ylabel('Test Score')
plt.title('Test Scores for Different Algorithms')
plt.ylim(0.7, 1.0)  # Set the y-axis limits for better visualization
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
pip install termcolor


In [ ]:
import pandas as pd
from tabulate import tabulate
from termcolor import colored

# Define the scores and models
# ... (your score definitions here)

# Create the DataFrame
models = pd.DataFrame({
    'Model': ['Logistic Regression',
              'KNN',
              'Naive Bayes',
              'Linear SVC',
              'Decision Tree',
              'Random Forest',
              'Gradient Booster',
              'XGBoost'
             ],
    'Train Score': [Logisticregressionscoretraining,
                    KNeighborsClassifierModelscoretraining,
                    GaussianNBModelscoretrain,
                    SVCModelscoretraining,
                    DecisionTreeClassifierModeltrain,
                    RandomForestClassifierModeltrain,
                    gb_clftrain,
                    xgb_train
                   ],
    'Test Score': [Logisticregressionscoretest,
                   KNeighborsClassifierModelscoretest,
                   GaussianNBModelscoretest,
                   SVCModelscoretest,
                   DecisionTreeClassifierModeltest,
                   RandomForestClassifierModeltest,
                   gb_clftest,
                   xgb_test
                  ]
})

# List of colors to cycle through
colors = ['red', 'green', 'blue', 'magenta', 'yellow', 'cyan', 'white']

# Prepare the formatted table rows
formatted_rows = []
for idx, row in models.iterrows():
    model_name = row['Model']
    train_score = row['Train Score']
    test_score = row['Test Score']
    
    color_idx = idx % len(colors)
    formatted_model = colored(model_name, colors[color_idx])
    formatted_row = [formatted_model, f'{train_score:.2f}', f'{test_score:.2f}']
    formatted_rows.append(formatted_row)

# Print the formatted table
headers = ['Model', 'Train Score', 'Test Score']
table = tabulate(formatted_rows, headers=headers, tablefmt='grid')
print(table)
